<a href="https://colab.research.google.com/github/nnilayy/Unet/blob/main/Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt 
import tensorflow as tf 
import numpy as np
import cv2 
import os 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Model Size


In [ ]:
IMAGE_SIZE=128

In [ ]:
!mkdir "/content/drive/MyDrive/dataset"

In [ ]:
!unzip -q /content/drive/MyDrive/dataset.zip -d /content/drive/MyDrive/

In [ ]:
train=ImageDataGenerator(
                        # rescale=1/255,
                         rotation_range=90,
                         brightness_range=(0,1.5),
                         zoom_range=0.2,
                        #  shear_range=0.2,
                         horizontal_flip=True,
                         vertical_flip=True,
                         fill_mode="nearest",
                         )
valid=ImageDataGenerator(
    # rescale=1/255,
                              rotation_range=90,
                              brightness_range=(0,1.5),
                              zoom_range=0.2,
                              # shear_range=0.2,
                              horizontal_flip=True,
                              vertical_flip=True,
                              fill_mode="nearest",)

In [ ]:
train_dataset=train.flow_from_directory('/content/drive/MyDrive/dataset_3/train',
                                        target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                        batch_size=5,
                                        class_mode='categorical',
                                        color_mode='grayscale',
                                        )
val_dataset=valid.flow_from_directory('/content/drive/MyDrive/dataset_3/validation',
                                        target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                        batch_size=5,
                                        class_mode='categorical',
                                        color_mode='grayscale'
                                        )

Found 301 images belonging to 2 classes.
Found 90 images belonging to 2 classes.


In [ ]:
val_dataset.class_indices

{'Cracked': 0, 'Not Cracked': 1}

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, AveragePooling2D, Flatten, Dropout, BatchNormalization, InputLayer, Reshape

model = Sequential()
model.add(InputLayer((IMAGE_SIZE, IMAGE_SIZE, 1)))
model.add(Conv2D(16, 3, padding='same', activation="relu"))
model.add(Conv2D(16, 3, padding='same', activation="relu"))
model.add(AveragePooling2D(2, padding='same'))
model.add(Conv2D(8, 3, padding='same', activation="relu"))
model.add(Conv2D(8, 3, padding='same', activation="relu"))
model.add(AveragePooling2D(2, padding='same'))
model.add(Conv2D(8, 3, padding='same', activation="relu"))
model.add(Conv2D(8, 3, padding='same', activation="relu"))
model.add(AveragePooling2D(2, padding='same'))
model.add(Flatten())
model.add(Dense(2, activation="softmax"))
model.summary()


# ---------------------------------------------------------------------------------------------------
# ---------------------------------------------------------------------------------------------------
# (1)Params: 32,000
# (2)Train_Accuracy: 85.45
# (3)Val_Accuracy: 84.44
# model = Sequential()
# model.add(InputLayer(input_shape=(224, 224,3)))
# model.add(Conv2D(16, 3, padding='same', activation="relu"))
# model.add(MaxPooling2D(2, padding='same'))
# model.add(Conv2D(16, 3, padding='same', activation="relu"))
# model.add(MaxPooling2D(2, padding='same'))
# model.add(Conv2D(16, 3, padding='same', activation="relu"))
# model.add(MaxPooling2D(2, padding='same'))
# model.add(MaxPooling2D(2, padding='same'))
# model.add(MaxPooling2D(2, padding='same'))
# model.add(Flatten())
# model.add(Dense(32, activation="relu"))
# model.add(Dense(1, activation="sigmoid"))

In [ ]:
model.compile(loss="binary_crossentropy", 
              optimizer=tf.keras.optimizers.Adamax(learning_rate=1e-3), 
              metrics=["accuracy"])

history=model.fit(train_dataset, 
                  steps_per_epoch=32, 
                  epochs=100, 
                  validation_data=val_dataset)

Epoch 1/100
32/32 [==============================] - 5s 53ms/step - loss: 0.9410 - accuracy: 0.4688 - val_loss: 0.7060 - val_accuracy: 0.4667
Epoch 2/100
32/32 [==============================] - 1s 40ms/step - loss: 0.6921 - accuracy: 0.5500 - val_loss: 0.6930 - val_accuracy: 0.4889
Epoch 3/100
32/32 [==============================] - 1s 38ms/step - loss: 0.6910 - accuracy: 0.5385 - val_loss: 0.6881 - val_accuracy: 0.5333
Epoch 4/100
32/32 [==============================] - 1s 38ms/step - loss: 0.6917 - accuracy: 0.5449 - val_loss: 0.6946 - val_accuracy: 0.5111
Epoch 5/100
32/32 [==============================] - 1s 39ms/step - loss: 0.6926 - accuracy: 0.5750 - val_loss: 0.6803 - val_accuracy: 0.6556
Epoch 6/100
32/32 [==============================] - 1s 39ms/step - loss: 0.6864 - accuracy: 0.5188 - val_loss: 0.6672 - val_accuracy: 0.6778
Epoch 7/100
32/32 [==============================] - 1s 39ms/step - loss: 0.6769 - accuracy: 0.6250 - val_loss: 0.6725 - val_accuracy: 0.6444
Epoch 

In [ ]:
model.save("/content/model_128.h5")

In [152]:
import cv2
file="/content/drive/MyDrive/dataset_3/train/Cracked/100.jpg"
image=cv2.imread(file, cv2.IMREAD_GRAYSCALE)
image=cv2.resize(image,(128,128))
image=np.expand_dims(image,0)
prediction=np.argmax(model.predict(image))

if prediction == 0:
  print('Cracked')

elif prediction==1:
  print('Not Cracked')

1/1 [==============================] - 0s 84ms/step
Cracked


In [ ]:
import os
def get_file_size(file_path):
  size=os.path.getsize(file_path)
  return size
def convert_bytes(size,unit=None):
  if unit=="KB":
    return print("File size: "+str(round(size/1024,3))+" Kilobytes")
  elif unit=="MB":
    return print("File size: "+str(round(size/(1024*1024),3))+" Megabytes")
  else:
    return print("File size: "+str(size)+"bytes")

In [ ]:
# Model Size Before Model Quantization
model="/content/model_128.h5"
convert_bytes(get_file_size(model),"MB")

File size: 0.061 Megabytes


In [ ]:
import glob
import cv2
import numpy as array 
files=glob.glob('/content/drive/MyDrive/dataset_3/train/Cracked/*')
images=[]
for file in files:
  image=cv2.imread(file, cv2.IMREAD_GRAYSCALE)
  image=cv2.resize(image,(256,256))
  image=image/255
  image=np.array(image,dtype=np.float32)
  images.append(image)

In [ ]:
images=np.array(images)
# images=np.squeeze(images,-1)
images.shape

In [ ]:
def dataset_generator():
   for i in range(100):
      input_image = tf.cast(images[i], tf.float32)
      input_image = tf.reshape(input_image, [1,256,256,1])
      yield ([input_image])

In [ ]:
import tensorflow as tf
model="/content/model_128.h5"
model=tf.keras.models.load_model(model)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations=[tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.representative_dataset=dataset_generator
tflite_model_name="model.tflite"
tflite_model = converter.convert()
open(tflite_model_name,"wb").write(tflite_model)

In [ ]:
# Model Size After Model Quantization
tflite_model="/content/model.tflite"
convert_bytes(get_file_size(tflite_model),"KB")

In [ ]:
import binascii
def convert_to_c_array(bytes) -> str:
  hexstr = binascii.hexlify(bytes).decode("UTF-8")
  hexstr = hexstr.upper()
  array = ["0x" + hexstr[i:i + 2] for i in range(0, len(hexstr), 2)]
  array = [array[i:i+10] for i in range(0, len(array), 10)]
  return ",\n  ".join([", ".join(e) for e in array])

tflite_binary = open('model.tflite', 'rb').read()
ascii_bytes = convert_to_c_array(tflite_binary)
header_file = "const unsigned char model_tflite[] = {\n  " + ascii_bytes + "\n};\nunsigned int model_tflite_len = " + str(len(tflite_binary)) + ";"
open("model_128.h", "w").write(header_file)
convert_bytes(get_file_size('/content/model_128.h'),"KB")

File size: 55.537 Kilobytes


In [ ]:
# train_data=train_dataset[0]
# X_train=train_data[0]
# y_train=train_data[1]

# test_data=val_dataset[0]
# X_test=test_data[0]
# y_test=test_data[1]

# print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)
# # print(y_test)

In [ ]:
for layer in model.layers:
  print(layer.get_config()['name'])

In [ ]:
# Convert the model to the TensorFlow Lite format without quantization
converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_TF)
model_no_quant_tflite = converter.convert()

# Save the model to disk
open(MODEL_NO_QUANT_TFLITE, "wb").write(model_no_quant_tflite)

# Convert the model to the TensorFlow Lite format with quantization
def representative_dataset():
  for i in range(500):
    yield([x_train[i].reshape(1, 1)])
# Set the optimization flag.
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# Enforce integer only quantization
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Provide a representative dataset to ensure we quantize correctly.
converter.representative_dataset = representative_dataset
model_tflite = converter.convert()

# Save the model to disk
open(MODEL_TFLITE, "wb").write(model_tflite)